In [1]:
#dataset import

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)



Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

#parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128


#number of samples to calculate validation and accuracy
test_valid_size = 256

#network parameters
n_classes = 10 # (0-9 digits)
dropout = 0.75 # dropout probability to keep units



> las imagenes son de 28x28

In [4]:
#Weights and biases
weights = {
    #hyperparametros, por los que se seleccionan el tamaño random (height y width)
    #el 1 es porque es en blanco y negro, si fuera de colores, serian 3 de depth
    #32 es hyperparametro
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    #la salida de la anterior, es el depth del siguiente, por eso tiene 32
    #el output depth siempre va creciendo
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
    #luego del droput el tamanio se reduce dos veces, por lo que llega a 7*7 por 
    #el depth del anterior, y se incrementa el output depth
    'wd1':tf.Variable(tf.random_normal([7*7*64,1024])),
    #salida
    'out':tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

In [5]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [6]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
    x,
    ksize=[1, k, k, 1],
    strides=[1, k, k, 1],
    padding='SAME'
    )

In [7]:
def conv_net(x, weights, biases, dropout):
    #Layer 1 - 25*28*1  pasa a ser de: 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool(conv1, k=2)
    
    #Layer 2 - 14*14*32 pasa a ser de: 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool(conv2, k=2)
    
    #Fully connected layer - 7*7*64 pasa a ser de 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    #output layer - class prediction - 1024 a 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [8]:
#tf graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

#model
logits = conv_net(x, weights, biases, keep_prob)

#define loss and optimizer
cost = tf.reduce_mean(\
                     tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
                     )

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)


#accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


#launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict = {
                x: batch_x,
                y: batch_y,
                keep_prob: dropout
            })
            
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.
            })
            
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob:1.
            })
            
            print('Epoch {:>2}, Batch{:>3} -'
                 'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                 epoch+1,
                 batch+1,
                 loss,
                 valid_acc
                 ))
            
            #test accuracy
            test_acc = sess.run(accuracy, feed_dict={
                x:mnist.test.images[:test_valid_size],
                y:mnist.test.labels[:test_valid_size],
                keep_prob:1.
            })
            
            print('Testing Accuracy: {}'.format(test_acc))
            
            

NameError: name 'maxpool' is not defined